In [18]:
import requests
import json

# Example ticket-tagging-api App usage.

This notebook shows how to use the ticket-tagging-api app.  It does not use the Steamship SDK to make it easier to port to other languages.

Each instance of the app has its own built-in context where examples are collected, labels are stored, etc.

## Prerequisites

* Using this assumes that you have already created an instance of the app (this is shown with the SDK in the app_instance_creation notebook.
* You must have the api_key associated with the deployed app instance.



## First, create a new App Instance

In this particular app, think of one "App Instance" as one "classifier" that you can specialize and use.

Use the `app_instance_creation.ipynb` to create an App Instance and then copy its `base_url` into the cell below:

In [19]:
#The base_url for the Steamship app, from the other notebook
base_url = 'https://assembled.steamship.run/default/ticket-tagging-api-20220720-zero-shot-b/'

In [20]:
print(base_url)

https://assembled.steamship.run/default/ticket-tagging-api-20220720-zero-shot-b/


In [42]:
# Your Steamship API key.
#
# This has either been provided to you, or you can run:
#
#    npm install -g @steamship/cli
#    ship login
#    ship user:info
#
# to view it.
api_key = ""

In [22]:
headers = {"Authorization": f"Bearer {api_key}"}

In [23]:
# Tiny wrapper for post calls of json data to the app.
def app_call(route: str, parameters: dict):
    return requests.post(base_url + route, headers=headers, json=parameters)

## You can always check the status of your classifier with `specialize_status`
* A response of `UNSPECIALIZED` means the app will classify data points using a large language model which doesn't need any training.
* A response of `SPECIALIZED` means the app will classify with a trained classifier based on a dataset you have provided.

In [24]:
# We haven't specialized our model yet, so this should return UNSPECIALIZED
response = app_call('specialize_status', {})
response.json()

{'status': 'UNSPECIALIZED'}

## First - a toy example for demonstrating the basic API

In [25]:
# clear any previous examples
app_call('clear_examples',{}).text

'Removed 0 examples'

In [26]:
# Set the labels we'll use for classification
response = app_call('set_labels',{'labels':['lion','tiger','elephant']})
response.text

'Labels accepted'

In [27]:
# Try a ticket classification (no training yet, so this is zero-shot)
response = app_call('tag_ticket',{'ticket_text':'I like big cats'})
response.json()

{'tiger': 0.6951338648796082,
 'lion': 0.6889975666999817,
 'elephant': 0.15381664037704468}

In [28]:
# Try a multi-ticket classification (no training yet, so this is zero-shot)
response = app_call('tag_tickets',{'ticket_texts':['I like big cats', 'Large gray animals with tusks are the best.']})
response.json()

[{'lion': 0.6889975666999817,
  'tiger': 0.6951338648796082,
  'elephant': 0.15381664037704468},
 {'lion': 0.7318058609962463,
  'tiger': 0.6491636037826538,
  'elephant': 0.5664064884185791}]

In [29]:
# Create some example tickets
examples = [('This is a ticket about lions.', ['lion']), ('This is a ticket about tigers.',['tiger']), ('This is a ticket about an elephant.',['elephant'])]

In [30]:
# Push the examples to the app
for example in examples:
    response = app_call('add_example', {'ticket_text':example[0], 'labels':example[1]})
    print(response.text)

Example accepted
Example accepted
Example accepted


In [31]:
# Push the examples to the app (in batch mode)
response = app_call('add_examples', {'ticket_texts':[example[0] for example in examples], 'labels':[example[1] for example in examples]})
print(response.text)

3 examples accepted


In [32]:
# How many examples have we added?
response = app_call('count_examples', {})
response.json()

{'total_examples': 6, 'label_examples': {'elephant': 2, 'lion': 2, 'tiger': 2}}

In [33]:
# We haven't specialized our model yet, so this should return UNSPECIALIZED
response = app_call('specialize_status', {})
response.json()

{'status': 'UNSPECIALIZED'}

In [34]:
# Remove the three toy examples
app_call('clear_examples',{}).text

'Removed 6 examples'

## Now, let's attempt with real data

In [35]:
# Read data adapted from test dataset, stripped to just the text to be
# tagged and lists of 'cleaned' tags (no underscores or leading or trailing spaces)
with open('test_data.json') as data_file:
    data = json.load(data_file)

len(data)

2133

In [36]:
# Find all unique tag labels
labels = list(set([label for row in data for label in row['tags']]))

In [37]:
# Set the app's labels
response = app_call('set_labels',{'labels':labels})
response.text

'Labels accepted'

In [38]:
# Tag an example ticket.  We haven't specialized yet, so this is zero-shot
response = app_call('tag_ticket',{'ticket_text':data[0]['text']})
response.json()

{'api': 0.8333170413970947,
 'needs review': 0.988425076007843,
 'bug': 0.5133237242698669,
 'automations': 0.10863839834928513,
 'non-support question': 0.7193129062652588,
 'security': 0.23934529721736908,
 'billing': 0.14298030734062195,
 'system credit card redaction': 0.06164781376719475,
 'time off': 0.06952499598264694,
 'realtime overview': 0.21536599099636078,
 'n/a': 0.049051713198423386,
 'alerts': 0.5230106711387634,
 'ticket mapping/queues': 0.07419535517692566,
 'datadog alerts support': 0.609268069267273,
 'inbound sales prospect': 0.24141553044319153,
 'setting': 0.8402145504951477,
 'closed by merge': 0.008600082248449326,
 'scheduling': 0.06945410370826721,
 'recruiting': 0.18673507869243622,
 'datadog alerts': 0.5168020725250244,
 'gcal': 0.6264351010322571,
 'not a support issue': 0.0005713383434340358,
 'self-served': 0.0029283815529197454,
 'alerts acknowledged': 0.9112120270729065,
 'web widget': 0.24844636023044586,
 'performance': 0.8535546660423279,
 'how do i

## Next lets specialize the model

There were not enough examples of all of the ticket types in the dataset, so we'll demonstrate here with only the top 10 ticket types.

In [39]:
top_10_tags = ['how do i',
 'n/a',
 'why isn t this working',
 'not a support issue',
 'datadog alerts support',
 'datadog alerts',
 'forecasting and staffing',
 'scheduling',
 'inbound sales prospect',
 'people']

In [40]:
# Set the labels in the app
response = app_call('set_labels',{'labels':top_10_tags})
response.text

'Labels accepted'

In [41]:
# Create dataset with only the top tags
top_tag_data = [dict(text=row['text'], tags=[tag for tag in row['tags'] if tag in top_10_tags]) for row in data] 

In [43]:
# Add examples to the app.
#
# NOTE:
# This cell will take a while to run, as we're adding each example one at a time.
for row in top_tag_data:
    response = app_call('add_example', {'ticket_text':row['text'], 'labels':row['tags']})
    print(response is not None)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [44]:
# Count the total examples per ticket.
# Note the total here is less than the total in the dataset, because after filtering to the top 10 tags, some examples don't have any tags anymore.
response = app_call('count_examples', {})
response.json()

{'label_examples': {'why isn t this working': 386,
  'not a support issue': 255,
  'n/a': 450,
  'how do i': 716,
  'forecasting and staffing': 191,
  'datadog alerts': 194,
  'scheduling': 191,
  'people': 188,
  'datadog alerts support': 194,
  'inbound sales prospect': 191},
 'total_examples': 1908}

In [45]:
# Start the specialization (training) process
#
# NOTE:
#  - This process will take up to FIVE HOURS for full training.
#  - During that time, you can continue using the app (via UNSPECIALIZED mode)
#  - While specializing, the status will be reported as: `SPECIALIZATION_IN_PROGRESS`
#
response = app_call('start_specialize',{})
response.text

'Started specialization. Please poll specialize_status'

In [47]:
# Check with the app and make sure training started successfully.
response = app_call('specialize_status', {})
response.json()


{'status': 'SPECIALIZATION_IN_PROGRESS'}

In [48]:
# Can still tag tickets with zero-shot while waiting
response = app_call('tag_ticket',{'ticket_text':'Another example ticket about Adherence'})
response.json()

{'how do i': 0.22574053704738617,
 'n/a': 0.0007463666843250394,
 'not a support issue': 0.001376710133627057,
 'why isn t this working': 0.00924806110560894,
 'forecasting and staffing': 0.00044003690709359944,
 'scheduling': 0.00024530277005396783,
 'inbound sales prospect': 0.01055943500250578,
 'datadog alerts support': 0.0009663257515057921,
 'datadog alerts': 0.0012334067141637206,
 'people': 0.007067074067890644}

In [528]:
# Now need to wait ~5 hours, or poll continuously
response = app_call('specialize_status', {})
response.json()

{'status': 'SPECIALIZATION_IN_PROGRESS'}

In [529]:
# Tag an example ticket.  We've specialized, so now we're using the trained model under the hood.
response = app_call('tag_ticket',{'ticket_text':data[0]['text']})
response.json()

{'scheduling': 0.06945410370826721,
 'datadog alerts': 0.5168020725250244,
 'not a support issue': 0.0005713383434340358,
 'inbound sales prospect': 0.24141553044319153,
 'n/a': 0.049051713198423386,
 'forecasting and staffing': 0.013277372345328331,
 'people': 0.7233816385269165,
 'how do i': 0.7680767178535461,
 'datadog alerts support': 0.609268069267273,
 'why isn t this working': 0.691179633140564}

In [390]:
data[0]['text']

'Hi Team, since we are still facing the issue of generating the previous months data for Adherence, can we request from you to send us the exported file for the month of November(in Agent Performance measuring adherence globally)?'